In [7]:
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

# Configurar la presión y el fluido
config = {"unit_pressure": "kPa"}
fluid = 'Water'

def update_plot(P1, P2, T3, rendimiento_bomba, rendimiento_turbina):
    # Datos conocidos
    P4 = P1  # presión en el punto 4 (igual a P1)
    P3 = P2  # presión en el punto 3 (igual a P2)
    
    # Propiedades en el punto 1
    s1 = CP.PropsSI('S', 'P', P1 * 1e6, 'Q', 0, fluid) * 1e-3
    h1 = CP.PropsSI('H', 'P', P1 * 1e6, 'Q', 0, fluid) * 1e-3
    T1 = CP.PropsSI('T', 'P', P1 * 1e6, 'Q', 0, fluid) - 273.15

    # Compresión isoentrópica por bombeo (etapa 1-2)
    v1 = 1 / CP.PropsSI('Dmass', 'P', P1 * 1e6, 'Q', 0, fluid)
    h2s = h1 + v1 * ((P2 - P1) * 1e3)  # Entalpía salida isentrópica de la bomba
    h2 = h1 - (h1 - h2s) / rendimiento_bomba # Entalpía salida real de la bomba
    s2 = CP.PropsSI("S", "P", P2 * 1e6, "H", h2 * 1e3, fluid) * 1e-3
    T2 = CP.PropsSI('T', 'P', P2 * 1e6, 'S', s2 * 1e3, fluid) - 273.15

    # Adición de calor en la caldera (etapa 2-3)
    h3 = CP.PropsSI('H', 'T', T3 + 273.15, 'P', P2 * 1e6, fluid) * 1e-3

    # Expansión en la turbina isoentrópica (etapa 3-4)
    s3 = CP.PropsSI('S', 'P', P2 * 1e6, 'H', h3 * 1e3, fluid) * 1e-3
    h4s = CP.PropsSI('H', 'P', P4 * 1e6, 'S', s3 * 1e3, fluid) * 1e-3  # Entalpía salida isentrópica de la turbina
    h4 = h3 - (h3 - h4s) * rendimiento_turbina  # Entalpía salida real de la turbina
    s4 = CP.PropsSI('S', 'P', P4 * 1e6, 'H', h4 * 1e3, fluid) * 1e-3
    s4s = s3
    T4 = CP.PropsSI('T', 'P', P4 * 1e6, 'S', s4 * 1e3, fluid) - 273.15
    T4s = CP.PropsSI('T', 'P', P4 * 1e6, 'S', s3 * 1e3, fluid) - 273.15

    # Calcular entropía y temperatura de saturación para P2
    s_liquid = CP.PropsSI('S', 'P', P2 * 1e6, 'Q', 0, fluid) * 1e-3
    s_vapor = CP.PropsSI('S', 'P', P2 * 1e6, 'Q', 1, fluid) * 1e-3
    T_liquid = CP.PropsSI('T', 'P', P2 * 1e6, 'Q', 0, fluid) - 273.15
    T_vapor = CP.PropsSI('T', 'P', P2 * 1e6, 'Q', 1, fluid) - 273.15
    
    # Calcular rendimiento del ciclo Rankine Simple
    rendimiento_rankine = (1 - (h4 - h1) / (h3 - h2)) * 100

    # Puntos del ciclo Rankine Simple
    se = [s1, s2, s3, s4, s1]
    Te = [T1, T2, T3, T4, T1]
    he = [h1, h2, h3, h4, h1]
    Pe = [P1, P2, P3, P4, P1]

    # Campana de vapor para T-s
    p_values = np.linspace(1, 22064, 1000)
    T_values = [CP.PropsSI('T', 'P', p * 1e3, 'Q', 0, fluid) - 273.15 for p in p_values]
    s_values_liquid = [CP.PropsSI('S', 'P', p * 1e3, 'Q', 0, fluid) * 1e-3 for p in p_values]
    s_values_vapor = [CP.PropsSI('S', 'P', p * 1e3, 'Q', 1, fluid) * 1e-3 for p in p_values]

    # Entalpía para líquido y vapor saturados
    h_values_liquid = [CP.PropsSI('H', 'P', p * 1e3, 'Q', 0, fluid) * 1e-3 for p in p_values]
    h_values_vapor = [CP.PropsSI('H', 'P', p * 1e3, 'Q', 1, fluid) * 1e-3 for p in p_values]

    # Presión para la campana de vapor
    P_values = [p * 1e-3 for p in p_values]

    # Crear el diagrama combinado T-s y P-h
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

    # Diagrama T-s
    ax1.set_title(f'Ciclo Rankine - Diagrama T-s (Rendimiento: {rendimiento_rankine:.2f}%)', color='b', fontsize=16)
    ax1.plot(s_values_liquid, T_values, 'b--', linewidth=2, label='Líquido Saturado')
    ax1.plot(s_values_vapor, T_values, 'r--', linewidth=2, label='Vapor Saturado')
    ax1.plot([s1, s2, s_liquid, s_vapor, s3, s4, s1], [T1, T2, T_liquid, T_vapor, T3, T4, T1], 'k', linewidth=3, label='Ciclo Rankine')
    ax1.plot([s2, s_liquid, s_vapor, s3], [T2, T_liquid, T_vapor, T3], 'y', linewidth=2, label='Isóbara de P2 (P3)')
    ax1.plot([s3, s4s], [T3, T4s], 'g--', linewidth=2, label='Isoentrópica de la turbina')

    # Ajustar límites de ejes
    ax1.set_xlim(min(s_values_liquid), max(s_values_vapor))
    ax1.set_ylim(min(T_values), T3 + 50)
    ax1.set_xlabel('Entropía (kJ/kg*K)', fontsize=14)
    ax1.set_ylabel('Temperatura (°C)', fontsize=14)
    ax1.legend()

    # Etiquetas numéricas con círculos en T-s
    ax1.annotate('1', (se[0], Te[0]), textcoords="offset points", xytext=(-10, 0), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax1.annotate('2', (se[1], Te[1]), textcoords="offset points", xytext=(-10, 15), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax1.annotate('3', (se[2], Te[2]), textcoords="offset points", xytext=(-2.5, 2.5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax1.annotate('4', (se[3], Te[3]), textcoords="offset points", xytext=(-10, 5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    
    # Diagrama P-h
    ax2.set_title(f'Ciclo Rankine - Diagrama P-h (Rendimiento: {rendimiento_rankine:.2f}%)', color='b', fontsize=16)
    ax2.plot(h_values_liquid, P_values, 'b--', linewidth=2, label='Líquido Saturado')
    ax2.plot(h_values_vapor, P_values, 'r--', linewidth=2, label='Vapor Saturado')
    ax2.plot(he, Pe, 'k', linewidth=3, label='Ciclo Rankine')
    ax2.plot([h3, h4s], [P2, P4], 'g--', linewidth=2, label='Isoentrópica de la turbina')

    # Ajustar límites de ejes
    ax2.set_xlim(min(h_values_liquid), h3 + 200)
    ax2.set_ylim(min(P_values), P2 + 1)
    ax2.set_xlabel('Entalpía (kJ/kg)', fontsize=14)
    ax2.set_ylabel('Presión (MPa)', fontsize=14)
    ax2.legend()

    # Etiquetas numéricas con círculos en P-h
    ax2.annotate('1', (he[0], Pe[0]), textcoords="offset points", xytext=(-10, 5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax2.annotate('2', (he[1], Pe[1]), textcoords="offset points", xytext=(-10, 5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax2.annotate('3', (he[2], Pe[2]), textcoords="offset points", xytext=(-5, 5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    ax2.annotate('4', (he[3], Pe[3]), textcoords="offset points", xytext=(-5, 5), ha='right', va='bottom',
              fontsize=12, fontweight='bold', color='k', bbox=dict(boxstyle="circle,pad=0.3", fc="white", ec="black", lw=2))
    
    # Mostrar los diagramas
    plt.tight_layout()
    plt.show()

# Definir sliders interactivos
P1_slider = widgets.FloatSlider(value=0.02, min=0.01, max=0.09, step=0.01, description='$P_1 (MPa)$')
P2_slider = widgets.FloatSlider(value=5, min=5, max=20, step=1, description='$P_2 (MPa)$')
T3_slider = widgets.FloatSlider(value=500, min=(CP.PropsSI('T', 'P', P2_slider.value * 1e6, 'Q', 0, fluid) - 273.15) + 100, max=1000, step=50, description='$T_3 (°C)$')
rendimiento_bomba_slider = widgets.FloatSlider(value=0.8, min=0.75, max=1.0, step=0.01, description='Rend. Bomba')
rendimiento_turbina_slider = widgets.FloatSlider(value=0.8, min=0.75, max=1.0, step=0.01, description='Rend. Turbina')

# Crear la interfaz interactiva
interactive_plot = interactive(update_plot, P1=P1_slider, P2=P2_slider, T3=T3_slider,
                               rendimiento_bomba=rendimiento_bomba_slider,
                               rendimiento_turbina=rendimiento_turbina_slider)

# Mostrar la interfaz interactiva
display(interactive_plot)

interactive(children=(FloatSlider(value=0.02, description='$P_1 (MPa)$', max=0.09, min=0.01, step=0.01), Float…